# Classification of bird sound files

<div class="alert alert-warning">
<strong>This notebook assume that the conversion of MP3 files into WAV files has been already done.</strong>

If not, please check the other notebook *birds_sound_visualizations.ipynb*
</div>

From literature, we have found several methods to classify sounds depending on the bird species:

- Logistic Regression
- k Nearest Neighbour (kNN) classifier with histogram-based features
- Support Vector Machines (SVM) with time-summarisation features
- Random Forest

We want a supervised classification, according to the litterature for bird sound SVM and KNN, due to their widespread use and ease of interpretation, will be the best method but it is interesting to compare them to other method thanks to an accuracy value.

The strategy here is to first create a dataframe with all the needed features from the bird sound recording that have been previously converted into wav file. Now that we have our labeled database of previously known bird songs, we use it to create a decision procedure that is used to predict the species of a new bird song.  

### Import libraries

In [1]:
import numpy as np
import os
import pandas as pd
from scipy.io import wavfile
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

### Create Dataframes

In [2]:
df_all = pd.read_csv('AllBirdsv4.csv')

# Create a new column to tell whether it's a blue pipit or not 
# this is our target for the classification
type=[]
for i in range(len(df_all.index)):
    if str(df_all.loc[i,'English_name']) == 'Rose-crested Blue Pipit':
        type.append(1)
    else: 
        type.append(0)
df_all['Type']=type
df_all_show = df_all.groupby('Type')
df_all_show = df_all_show.apply(lambda x: x.sample(frac=0.3))
display(df_all_show)

File ID                English_name Vocalization_type   Quality  \
Type                                                                         
0    1383    81094              Ordinary Snape              call         C   
     1278   163249          Orange Pine Plover              call         C   
     1485   162951         Purple Tooting Tout              call         A   
     1157   372260          Orange Pine Plover        call, song         B   
     1992     5674                 Scrawny Jay              Call         C   
     2073    17101         Vermillion Trillian              Song         C   
     570    397437     Carries Champagne Pipit              call  no score   
     1043   103452           Lesser Birchbeere              call         B   
     1437   101888                   Pinkfinch              call         B   
     532    171991           Canadian Cootamum        call, song         B   
     858    340294  Green-tipped Scarlet Pipit              song         A   
     739    246976            Darkwing Sparrow                 ?         A   
     1891    28249          Bent-beak Riffraff              song         B   
     421    395415           Broad-winged Jojo              call         B   
     1312   259259              Ordinary Snape              call         A   
     553    181303           Canadian Cootamum              call         C   
     1164   360933          Orange Pine Plover              song         B   
     1149   390273          Orange Pine Plover              song         B   
     1690   253987                  Queenscoat              song         B   
     2070    89162         Vermillion Trillian              call         C   
     989     34783           Lesser Birchbeere              call         A   
     1757   401071                  Queenscoat              call         C   
     1845    88362          Bent-beak Riffraff              Song         A   
     1854    30317          Bent-beak Riffraff              song         A   
     1346   302960              Ordinary Snape              call         B   
     593     61943     Carries Champagne Pipit              call         A   
     2010   139430         Vermillion Trillian        call, song         A   
     1189   287307          Orange Pine Plover              call         B   
     1011   190805           Lesser Birchbeere              call         B   
     198    239019        Blue-collared Zipper              call         A   
...            ...                         ...               ...       ...   
1    24     164501     Rose-crested Blue Pipit              song         A   
     8      377874     Rose-crested Blue Pipit              song         A   
     64     345795     Rose-crested Blue Pipit              call         B   
     28     162567     Rose-crested Blue Pipit              call         A   
     158    277998     Rose-crested Blue Pipit              song         C   
     21     225347     Rose-crested Blue Pipit              call         A   
     178     18969     Rose-crested Blue Pipit              song         C   
     107    233384     Rose-crested Blue Pipit              song         B   
     88     313778     Rose-crested Blue Pipit              song         B   
     145     16969     Rose-crested Blue Pipit              Song         B   
     12     353709     Rose-crested Blue Pipit              call         A   
     132    125878     Rose-crested Blue Pipit              call         B   
     185    306340     Rose-crested Blue Pipit              song         E   
     47      41283     Rose-crested Blue Pipit              call         A   
     60     345802     Rose-crested Blue Pipit              song         B   
     31     162563     Rose-crested Blue Pipit              song         A   
     141     53789     Rose-crested Blue Pipit              song         B   
     86     320496     Rose-crested Blue Pipit              song         B   
     53     387403     Rose-crest

In [3]:
# Read all sounds files
folder_path = 'all_birds_wav'
files = os.listdir(folder_path)

sounds_all = []
for file_id in df_all_show['File ID']: 
    # Read WAV file
    f = os.path.join(folder_path, "%s.wav" % (file_id))
    rate, data = wavfile.read(f)
    sounds_all.append(data[:500000])

In [4]:
# Read the tests sounds of Kasios
folder_path = 'test_birds_kasios_wav'
files = os.listdir(folder_path)

sounds_test = []
for file in files:
    f = os.path.join(folder_path, file)
    rate, data = wavfile.read(f)
    sounds_test.append(data[:500000])

In [5]:
# Compute 4 parameters for all sounds
mean_all = []
std_all = []
min_all = []
max_all = []

for sound in sounds_all:
    mean_all.append(np.mean(sound))
    std_all.append(np.std(sound))
    min_all.append(np.min(sound))
    max_all.append(np.max(sound))

df_all_show = df_all_show.assign(mean=mean_all, std=std_all, 
                                 min=min_all, max=max_all)

df_all_show.head()

File ID         English_name Vocalization_type Quality    Time  \
Type                                                                        
0    1383    81094       Ordinary Snape              call       C  6:30am   
     1278   163249   Orange Pine Plover              call       C   10:00   
     1485   162951  Purple Tooting Tout              call       A   12:00   
     1157   372260   Orange Pine Plover        call, song       B   11:00   
     1992     5674          Scrawny Jay              Call       C       ?   

                 Date    X    Y  Type        mean          std    min    max  
Type                                                                          
0    1383    6/8/2011  124  110     0    0.587196  1126.260367 -13927  14539  
     1278  12/26/2012  105   77     0   -0.261142   644.658387  -5608   6998  
     1485  12/10/2013   67   95     0    0.008420   628.725888 -16340  15585  
     1157   5/27/2017  142  113     0    0.823518   186.947862  -1425   1683  
     1992  11/13/1992   50   81     0  106.810938   914.084886 -12920  11746

In [6]:
# Compute 4 parameters for test sounds
df_test = pd.read_csv('Test Birds Location.csv')
mean_test = []
std_test = []
min_test = []
max_test = []

for sound in sounds_test:
    mean_test.append(np.mean(sound))
    std_test.append(np.std(sound))
    min_test.append(np.min(sound))
    max_test.append(np.max(sound))

df_test = df_test.assign(mean=mean_test, std=std_test, 
                         min=min_test, max=max_test)

df_test = df_test.drop(columns=['ID'])

df_test.head()

,X,Y,mean,std,min,max
0,140,119,-0.072540,794.113556,-6766,9935
1,63,153,-0.199194,877.144848,-12501,12258
2,70,136,-0.405632,957.414030,-11614,14137
3,78,150,-0.073872,685.782020,-8716,8549
4,60,90,-0.306266,1549.637434,-12257,11579


### k Nearest Neighbour (k-NN) classifier

#### 1) Training Model

In [7]:
X_sample = df_all_show.drop(columns=['Quality', 'English_name', 
                                     'Vocalization_type', 'Time', 'Date', 
                                     'File ID', 'Type'])
X_sample["Y"] = X_sample["Y"].map(lambda y: y.replace("?",""))
X_sample['Y'] = X_sample['Y'].astype(float)

X = []

y = list(df_all_show["Type"])

for row in X_sample.iterrows():
    index, data = row
    X.append(data.tolist())

In [8]:
# k-NN training
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
clf = KNeighborsClassifier(n_neighbors=2)
clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=2, p=2,
           weights='uniform')

In [9]:
# Accuracy of the model
accuracy = clf.score(X_test, y_test)
print("Accuracy of the model: %s" % accuracy)

# Confusion matrix
y_predict= clf.predict(X_test)
print("Confusion matrix: \n%s" % confusion_matrix(y_test, y_predict))

Accuracy of the model: 0.88
Confusion matrix: 
[[110   0]
 [ 15   0]]


#### 2) Predicting on Kasios test sounds 

In [10]:
X_kasios = []

for row in df_test.iterrows():
    index, data = row
    X_kasios.append(data.tolist())

In [11]:
kasios_prediction = clf.predict(X_kasios)
for i in range(len(kasios_prediction)):
    print("Sound test %s - Prediction : %s" % (i+1, kasios_prediction[i]))

Sound test 1 - Prediction : 0
Sound test 2 - Prediction : 0
Sound test 3 - Prediction : 0
Sound test 4 - Prediction : 0
Sound test 5 - Prediction : 0
Sound test 6 - Prediction : 0
Sound test 7 - Prediction : 0
Sound test 8 - Prediction : 0
Sound test 9 - Prediction : 0
Sound test 10 - Prediction : 0
Sound test 11 - Prediction : 0
Sound test 12 - Prediction : 0
Sound test 13 - Prediction : 0
Sound test 14 - Prediction : 0
Sound test 15 - Prediction : 0


<div class='alert alert-info'>
According to our <b>k-NN classifier</b>, the 15 Kasios test sounds <b>do not seem</b> to be Blue Pipit one.
<div>  

### Support Vector Machine (SVM) classifier

#### 1) Training Model

In [ ]:
x_list = np.linspace(2**-5,2**5,30)
accuracy_C =[]
for x in x_list:
    model = svm.SVC(kernel='linear', C=x)
    model.fit(X_train, y_train)
    accuracy_C.append(model.score(X_test,y_test))
x=0

# to show the plot to be more visual
plt.plot(x_list,accuracy_C,'ro')
plt.show()
max_index = accuracy_C.index(max(accuracy_C)) #to find the C with the max accuracy
C_chosed = x_list[max_index] #the C that we will choose

In [ ]:
#we do the same thing for gamma

accuracy_gamma =[]
for x in x_list:
    model = svm.SVC(kernel='linear', C=C_chosed, gamma=x)
    model.fit(X_train, y_train)
    accuracy_gamma.append(model.score(X_test,y_test))

plt.plot(x_list,accuracy_gamma,'ro')
plt.show()

max_index = accuracy_gamma.index(max(accuracy_gamma)) #to find the C with the max accuracy
gamma_chosed = x_list[max_index] #the C that we will choose

In [ ]:
#our svm is then
model = svm.SVC(kernel='linear', C=C_chosed, gamma=gamma_chosed)
model.fit(X_train, y_train)

In [ ]:
# Accuracy of the model
accuracy = model.score(X_test, y_test)
print("Accuracy of the model: %s" % accuracy)

# Confusion matrix
y_predict= model.predict(X_test)
print("Confusion matrix: \n%s" % confusion_matrix(y_test, y_predict))

#### 2) Predicting on Kasios test sounds 

In [ ]:
kasios_prediction = model.predict(X_kasios)
for i in range(len(kasios_prediction)):
    print("Sound test %s - Prediction : %s" % (i+1, kasios_prediction[i]))

<div class='alert alert-info'>
According to our <b>SVM classifier</b>, the 15 Kasios test sounds <b>do not seem</b> to be Rose crested Blue Pipit's.
<div>  